In [6]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%autoreload 2

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path

In [9]:
path = Path('/app/data/raw/fossil_infrastructure.xlsx')
assert path.exists()

In [11]:
# eip = dbcp.extract.eip_infrastructure.extract(path)
# hardcode the extract function so this notebook can be easily rerun in the future without maintenance
sheets_to_read = [
    'Facility',
    'Company',
    'Project',
    'Air Construction',  # permit status is key to identifying actionable projects
    #'Pipelines',
]
eip = pd.read_excel(path, sheet_name=sheets_to_read)
rename_dict = {
    'Facility': 'eip_facilities',
    'Company': 'eip_companies',
    'Project': 'eip_projects',
    'Air Construction': 'eip_air_constr_permits',
    #'Pipelines': 'eip_pipelines',
}
eip = {rename_dict[key]: df for key, df in eip.items()}

In [12]:
eip.keys()

dict_keys(['eip_facilities', 'eip_companies', 'eip_projects', 'eip_air_constr_permits'])

In [13]:
{k: df.shape for k, df in eip.items()}

{'eip_facilities': (622, 50),
 'eip_companies': (531, 14),
 'eip_projects': (768, 39),
 'eip_air_constr_permits': (879, 16)}

In [14]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [16]:
fac = eip['eip_facilities']
cos = eip['eip_companies']
proj = eip['eip_projects']
air = eip['eip_air_constr_permits']
#pipe = eip['eip_pipelines']

Outline of work
Two parts: data cleaning and data normalization/structuring
# Structuring and Normalizaing
**5 entities and 5 many:many relationships means 10 tables...** But utting both pipelines and companies drops the total tables in half to 5.

The *only* purpose of bringing in the companies table is to add one column with ownership info. But the marginal cost is 3 tables (2 if cutting pipelines), or 30% of tables. I'll confirm with DBCP that this is OK.

Pipelines are approved at the federal level so I'm not sure they are actionable for Down Ballot people. They also have only very coarse location information (state). We punted on them last time so I would like to do so again. Marginal cost is also 3 tables, or 2 additional if cutting companies.

## Entity Relationships
### Entities
* facilities
* companies
* projects
* permits (air construction permits. there are many other permit types that I didn't integrate)
* pipelines

### Relationships
many : many
* facilities : companies
* facilities : projects
* facilities : pipelines
* companies : pipelines
* projects : permits

one : many
* none

one : one
* none

no direct relationship
* facilities : permits (air construction permits are mediated through projects. Other permits not considered here do have direct relationships)
* companies : projects (mediated through facilities)
* companies : permits (mediated through projects then through facilities)
* projects : pipelines (mediated through facilities)
* permits : pipelines (mediated through projects then through facilities)

# Cleaning
Need to clean facilities, projects, and permits via the usual checklist. But I can ignore many unecessary columns and prefix them 'raw_' to discourage use.
## Facilities Cleaning
- [x] Accuracy
- [x] Atomicity
- [ ] Consistency
- [x] Completeness
- [x] Uniformity
- [x] Validity
    - [x] Range Validation
    - [x] Uniqueness Validation
    - [x] Set Membership Validation
    - [x] Type Validation
    - [x] Cross-Field Validation

### Accuracy
I'm mostly using this table for location information, so I'll focus on the "street address" and "coordinates" columns. I don't have "golden data" to compare against, but I can at least spot check some items by googling them. \[Update: 3/3 spot checks of location are good. Obviously this is far from comprehensive but gives a small measure of confidence.]

In [17]:
fac.sample(3, random_state=42)

,id,name,created_on,modified_on,CCS/CCUS,CCS (ID),CCS,Company (ID),Company,Project (ID),Project,State,Facility Alias,Facility Description,Latest Updates,State Facility ID Number(s),Primary NAICS Code,Primary SIC Code,Street Address,City,ZIP Code,County or Parish,Associated Facilities (ID),Associated Facilities,Pipelines (ID),Pipelines,Air Operating (ID),Air Operating,CWA-NPDES (ID),CWA-NPDES,CWA Wetland (ID),CWA Wetland,Other Permits (ID),Other Permits,Congressional Representatives,Link to EJSCREEN Report,Estimated Population within 3 miles,Percent People of Color within 3 miles,Percent Low-Income within 3 miles,Percent under 5 Years Old within 3 miles,Percent People over 64 Years Old within 3 miles,Air Toxics Cancer Risk (NATA Cancer Risk),Respiratory Hazard Index,PM2.5 (ug/m3),O3 (ppb),Wastewater Discharge Indicator,Location,Facility Footprint,EPA FRS ID,Facility ID
249,995,Nutrien Lima Nitrogen Plant,2021-05-20T19:13:45.411472,2022-08-24T16:50:19.743634,NaN,NaN,NaN,2635,"PCS Nitrogen Ohio, L.P.[2635]",3002,Nutrien Lima Nitrogen Plant Expansion[3002],OH,NaN,The PCS Lima Nitrogen Plant is a nitrogen fert...,NaN,03-02-02-0370,NaN,NaN,2200 Ft Amanda Rd,Lima,45804,Allen,NaN,NaN,NaN,NaN,5424,P0128100[5424],NaN,NaN,NaN,NaN,NaN,NaN,"Jim Jordan, Republican",{u'value': u'https://ejscreen.epa.gov/mapper/E...,35287.0,31.0,42.0,6.0,16.0,28.0,0.3,9.12,45.7,0.0270,"-84.136881, 40.710166",NaN,NaN,10279.0
399,3752,Fort Dodge Nitrogen Plant,2021-09-01T18:07:47.558580,2022-08-29T23:13:23.199404,NaN,NaN,NaN,2577,"Koch Nitrogen Company, LLC[2577]",5431,UAN Expansion Project[5431],IA,NaN,This facility produces anhydrous nitrogen and ...,"On Aug. 18, 2022, Koch Fertilizers announced a...",94-01-005,325311,2873,3162 200th St.,Fort Dodge,50501,Webster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{u'value': u'https://ejscreen.epa.gov/mapper/E...,485.0,1.0,33.0,2.0,15.0,20.0,0.2,8.02,41.0,0.0019,"-94.019327, 42.499648",NaN,NaN,NaN
174,916,Jackson Compressor Station,2021-05-20T19:13:45.411472,2021-10-29T17:05:42.658809,NaN,NaN,NaN,2519,"Empire Pipeline, Inc.[2519]",2918,Jackson Compressor Station[2918],PA,Jackson Compressor Station-Empire North Expans...,NaN,NaN,827160,NaN,NaN,3438 Buckwheat Hollow Road,Lawrenceville,16929,Tioga,NaN,NaN,3291,Empire North Expansion Project[3291],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Fred Keller, Republican",{u'value': u'https://ejscreen.epa.gov/mapper/E...,1148.0,2.0,27.0,7.0,18.0,20.0,0.2,5.64,38.4,0.0041,"-77.031564, 41.998566",NaN,NaN,10140.0


Googling "Oak Grove Gas Plant" turns up the facility. [Street address](https://www.google.com/maps/place/Williams+Natural+Gas+Oak+Grove+Facility/@39.871189,-80.6944623,1177m/data=!3m1!1e3!4m13!1m7!3m6!1s0x0:0x6769abd010d373f9!2zMznCsDUyJzMyLjkiTiA4MMKwNDEnNDUuMiJX!3b1!8m2!3d39.8758!4d-80.6959!3m4!1s0x8835e69402fb74cd:0x94b44b7720f51c5!8m2!3d39.8690544!4d-80.693195) and coordinates match. Owner also matches.

MarkWest Houston Complex location is also good. Google maps labels the [corporate office](https://www.google.com/maps/place/MarkWest+Houston+Plant/@40.262237,-80.2596898,1240m/data=!3m1!1e3!4m13!1m7!3m6!1s0x8834528cbcacb571:0xbd8b49797f3fdd4!2s800+Western+Ave,+Washington,+PA+15301!3b1!8m2!3d40.2584361!4d-80.2555021!3m4!1s0x8834539d500f0e45:0x248d758337e3de37!8m2!3d40.2585062!4d-80.254957) as across the street from the given address, which belongs to a different facility building. But that doesn't matter for our purposes -- we aren't sending them a letter. Owner also matches.

Formosa Point Comfort plant street address matches [google maps](https://www.google.com/maps/place/Formosa+Plastics+Corporation,+Texas/@28.6804226,-96.5626898,13964m/data=!3m1!1e3!4m5!3m4!1s0x0:0x469e4fbb5f6d12a1!8m2!3d28.6975144!4d-96.5449333) and coordinates are inside the facility. Owner also matches.

### Atomicity
By inspection I see that all the ID and associated name fields can contain multiple values: company, project, pipelines, and permits. The location fields are mercifully single valued

In [18]:
# street address does not look multi-valued but has other problems. Thankfully lat lon is still available
# a little more digging suggests bad addresses are because these have not yet been built.
# Can't check for sure until I can join project status on to facilities
pd.options.display.max_colwidth = 0
fac.loc[fac['Street Address'].str.len().nlargest(10).index, ['id', 'name', 'Street Address', 'Location']]

,id,name,Street Address,Location
461,3839,Rio Bravo Compressor Station 2,From intersection of I69E N and US 77 N turn left onto Unnamed Rd. Go 1.5 mi site on R,"-97.786294, 26.609886"
11,750,Annova LNG Brownsville,USFWS Access Road (left from intersection of Boca Chica Blvd and Kingston Ave),"-97.2675, 26.00556"
356,1105,Turkey Creek Compressor Station,W on Onyx Rd (towards the intersection of Johnsons Landing Rd),"-92.424444, 30.939722"
284,1031,El Paso Natural Gas - Red Mountain Compressor Station,1.4 miles on Co Rd D0006 from the intersection with NM-418,"-107.998849, 32.257081"
515,4480,Lone Star Alkylate Production Facility,Approx. 1.8 miles SW from FM 1942 and Hatcherville Rd,"-94.923882, 29.84787"
89,829,Corpus Christi Polymer & Desalination Plant,7001 Joe Fulton International Trade Corridor STE 200,"-97.49595, 27.834238"
374,1124,Willcox and Dragoon Compressor Stations,Arzberger Rd (6 miles E of Kansas Settlement Rd),"-109.662345, 32.109089"
99,839,Delta LNG Terminal,LA Hwy 23 (22 mi S from West Pointe à la Hache),"-89.873677, 29.596179"
496,4316,Dos Picos Gas Plant,CR 1090 (approx. 20 mi SE from I20 and Hwy 158),"-101.86781, 31.88272"
272,1019,Port Arthur LNG Export Terminal,State Hwy 87 (5.3 mi from State Hwy 82 S),"-93.958604, 29.786741"


In [19]:
# location is not multi-valued - exactly two decimal points per coordinate pair
fac['Location'].str.count('\.').agg(['min', 'max'])

min    2.0
max    2.0
Name: Location, dtype: float64

In [20]:
# a shitload of missing facility IDs, but no multi-valued ones
fac['Facility ID'].describe()

count    383.000000  
mean     10195.493473
std      113.342341  
min      10000.000000
25%      10097.500000
50%      10195.000000
75%      10293.500000
max      10393.000000
Name: Facility ID, dtype: float64

### Completeness
Notable missing values and lack of missing values:
* 93/563 (16.5%) missing street address. Plus some addresses are not missing but look unusable.
* 4/563 (0.7%) of facilities are missing linked Project IDs
* 9/563 (1.6%) missing "Location" (coordinates)
* 3/563 (tiny%) missing county (none missing state). But the true test is how successful `addfips` is with these pairs
* 60 to 100 (10% to 18%) missing EJ Screen metrics, depending on which metric

I don't know what `Facility ID` is (vs `id` of this facility table), but 179/563 (31.8%) rows are missing `Facility ID`. They have different numerical ranges and I see that the companies and project tables thankfully use the `id` numbers, which are 100% complete.

Based on these nan counts, I should first try `addfips` on state/county pairs. If too many fail, the most complete option is to geocode via coordinates.

In [21]:
len(fac)

622

In [22]:
fac.count().T

id                                                 622
name                                               622
created_on                                         622
modified_on                                        622
CCS/CCUS                                           33 
CCS (ID)                                           29 
CCS                                                29 
Company (ID)                                       618
Company                                            618
Project (ID)                                       617
Project                                            617
State                                              621
Facility Alias                                     182
Facility Description                               457
Latest Updates                                     31 
State Facility ID Number(s)                        489
Primary NAICS Code                                 236
Primary SIC Code                                   219
Street Add

### Consistency - defer
Defer until I've cleaned the related datasets
### Uniformity
Important columns to check consistent representation:
* coordinates
* ID fields (check consistent array delimiters)

Secondary importance:
* street address (this is a luxury field)
* modified_on

#### Coordinates

In [23]:
# "-XX.X, YY.Y" with 2 or 3 digits before the decimal and 2 to 7 digits after.
# Plus optional leading/trailing whitespace.
coord_pattern = r'\s*-\d{2,3}\.\d{2,7}, \d{2,3}\.\d{2,7}\s*'
fac['Location'].str.match(coord_pattern).sum()

606

In [24]:
# matches count, so they all have the same formatting
fac['Location'].count()

606

In [25]:
# tighten criteria to 3+ digits after decimal
# Reveals that only 2 facilities have poor precision (plus or minus about a km)
coord_pattern = r'\s*-\d{2,3}\.\d{3,7}, \d{2,3}\.\d{3,7}\s*'
fac['Location'].str.match(coord_pattern).sum()

604

#### ID Fields
Want to check for consistent array delimiters.

In [26]:
# exclude ID cols with numeric types (no arrays present)
id_cols = [col for col in fac.columns if '(ID)' in col and pd.api.types.is_object_dtype(fac[col])]
id_cols

['Company (ID)',
 'Project (ID)',
 'Associated Facilities (ID)',
 'Pipelines (ID)',
 'Air Operating (ID)',
 'CWA-NPDES (ID)',
 'Other Permits (ID)']

In [27]:
# mandatory opening pattern, optional delimiter, optional repeating pattern, optional closing pattern, mandatory end of line
array_pattern = r'(?:\d{3,5})(?:, ?)?(?:\d{3,5}, ?)*(?:\d{3,5})?$'

In [28]:
test_case = pd.Series([
    '1234',
    '1234,567',
    '1234, 567',
    '12345, 678, 9012',
    '1234\t5678', # tab is bad, no comma
    '12, 3456', # too short
    '1234    5678', # too many spaces, no comma
])
pd.concat([test_case, test_case.str.match(array_pattern)], axis=1)

,0,1
0,1234,True
1,"1234,567",True
2,"1234, 567",True
3,"12345, 678, 9012",True
4,1234\t5678,False
5,"12, 3456",False
6,1234 5678,False


In [29]:
# all pass the formatting test
for col in id_cols:
    assert fac[col].str.match(array_pattern).all()

#### Date Modified

In [30]:
# to_datetime works on all values present
timestamps = pd.to_datetime(fac['modified_on'])
timestamps.dtypes, timestamps.isna().sum()

(dtype('<M8[ns]'), 0)

#### Street Address - defer
hard to test and I don't care that much if it's wrong. Best way to test is probably to outsource to a pre-built geocoder

### Range Validation
Check IDs and Coordinates
#### Coordinates
All the extreme coordinates are real places! No "Null Island" dwellers either.

In [31]:
coords = fac['Location'].str.split(',', expand=True)
for col in coords.columns:
    coords.loc[:, col] = pd.to_numeric(coords.loc[:, col], errors='coerce')
coords.head()

,0,1
0,-103.525728,32.542358
1,-80.380335,40.331198
2,-105.779270,43.851240
3,-101.422777,35.641666
4,-84.250549,31.541712


In [32]:
coords.describe()

,0,1
count,606.000000,606.000000
mean,-93.215072,35.597120
std,13.602131,7.428577
min,-151.383102,17.710307
25%,-97.471682,29.990337
50%,-93.325802,32.436405
75%,-81.732684,39.981558
max,-64.754109,70.319900


In [33]:
# look at extreme coordinates
# max longitude
fac.loc[coords[0].idxmax(), ['Location', 'City', 'ZIP Code', 'Facility Description']]

Location                -64.754109, 17.710307                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
City                    St Croix                                                                                                                                                          

In [34]:
# Min longitude
fac.loc[coords[0].idxmin(), ['Location', 'City', 'ZIP Code', 'Facility Description']]

Location                -151.383102, 60.680077                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
City                    Kenai                                                                                                                                                                                                                                                                                                                                                                                           

In [35]:
# max latitude
fac.loc[coords[1].idxmax(), ['Location', 'City', 'ZIP Code', 'Facility Description']]

Location                -148.5573, 70.3199                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [36]:
# Min Latitude
fac.loc[coords[1].idxmin(), ['Location', 'City', 'ZIP Code', 'Facility Description']]

Location                -64.754109, 17.710307                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
City                    St Croix                                                                                                                                                          

#### IDs
There are lots of ID columns, but I only care about project IDs and associated facilities

In [37]:
# defined way up near the top
id_cols

['Company (ID)',
 'Project (ID)',
 'Associated Facilities (ID)',
 'Pipelines (ID)',
 'Air Operating (ID)',
 'CWA-NPDES (ID)',
 'Other Permits (ID)']

In [38]:
proj_ids = fac['Project (ID)'].str.split(',', expand=True)
for col in proj_ids.columns:
    proj_ids.loc[:, col] = pd.to_numeric(proj_ids.loc[:, col], errors='coerce')

proj_ids.head()

,0,1,2,3,4,5,6,7,8,9
0,2723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2724.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2725.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2729.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# they all look in the same range
proj_ids.describe()

,0,1,2,3,4,5,6,7,8,9
count,617.000000,95.000000,29.000000,15.000000,12.000000,3.000000,1.0,1.0,1.0,1.0
mean,3529.740681,3998.273684,3942.862069,4343.466667,4399.166667,4962.666667,3663.0,3664.0,3665.0,4258.0
std,856.325828,880.629012,877.249383,1047.621787,751.985957,700.665636,NaN,NaN,NaN,NaN
min,2723.000000,2732.000000,2836.000000,2855.000000,3090.000000,4161.000000,3663.0,3664.0,3665.0,4258.0
25%,2897.000000,3058.500000,3082.000000,3404.000000,3933.250000,4715.000000,3663.0,3664.0,3665.0,4258.0
50%,3070.000000,4143.000000,3987.000000,4309.000000,4233.500000,5269.000000,3663.0,3664.0,3665.0,4258.0
75%,3976.000000,4667.000000,4403.000000,5334.000000,5087.000000,5363.500000,3663.0,3664.0,3665.0,4258.0
max,5502.000000,5492.000000,5382.000000,5451.000000,5311.000000,5458.000000,3663.0,3664.0,3665.0,4258.0


In [40]:
assoc_ids = fac['Associated Facilities (ID)'].str.split(',', expand=True)
for col in assoc_ids.columns:
    assoc_ids.loc[:, col] = pd.to_numeric(assoc_ids.loc[:, col], errors='coerce')

assoc_ids.head()

,0,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# they all look in the same range
assoc_ids.describe()

,0,1,2,3,4,5,6,7,8,9,10
count,180.000000,102.000000,45.000000,12.000000,12.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1.0
mean,2237.188889,2636.343137,2500.533333,1278.166667,967.583333,949.181818,952.636364,969.545455,1009.727273,1061.545455,1016.0
std,1729.658841,1729.852054,1591.369887,1076.842255,93.502998,90.300408,71.855790,62.311096,41.499617,45.204787,NaN
min,756.000000,754.000000,755.000000,871.000000,818.000000,808.000000,871.000000,871.000000,940.000000,966.000000,1016.0
25%,869.500000,882.250000,908.000000,908.000000,894.500000,875.000000,904.500000,940.000000,966.000000,1027.000000,1016.0
50%,1019.000000,3626.000000,3778.000000,995.500000,982.500000,908.000000,940.000000,966.000000,1025.000000,1097.000000,1016.0
75%,3998.000000,4140.500000,3992.000000,1027.000000,1027.000000,1037.500000,982.500000,995.500000,1048.000000,1097.000000,1016.0
max,5500.000000,5496.000000,4689.000000,4692.000000,1097.000000,1048.000000,1097.000000,1097.000000,1048.000000,1097.000000,1016.0


### Uniqueness Validation
Check the `id` field (NOT `Facility ID`)

In [42]:
fac['id'].duplicated().sum()

0

### Set Membership Validation
Check state/county only. A few takeaways:
* state 'TDB' values need conversion to NULL
* a few states are arrays (but only one value, duplicated)
* 4 counties are arrays. Probably just take the first one. The better but more expensive way is to use the given lat, lon coords. Not worth it for 4 facilities.

In [43]:
from pudl.helpers import add_fips_ids

/app/.local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [44]:
w_fips = add_fips_ids(fac[['State', 'County or Parish']], state_col='State', county_col='County or Parish', vintage=2020)
w_fips.head()

,State,County or Parish,state_id_fips,county_id_fips
0,NM,Lea,35,35025
1,PA,Washington,42,42125
2,WY,Campbell,56,56005
3,TX,Hutchison,48,<NA>
4,GA,Dougherty,13,13095


In [45]:
w_fips.shape

(622, 4)

In [46]:
# 9 bad states and 20 bad state/county combos
w_fips.describe()

,State,County or Parish,state_id_fips,county_id_fips
count,621,613,617,593
unique,47,279,44,283
top,TX,Jefferson,48,48245
freq,173,28,173,24


In [47]:
w_fips[w_fips['county_id_fips'].isna()]

,State,County or Parish,state_id_fips,county_id_fips
3,TX,Hutchison,48,<NA>
32,LA,<NA>,22,<NA>
135,LA,"Ascension, Iberville",22,<NA>
195,VI,St. Croix,78,<NA>
247,FL,Gulf of Mexico,12,<NA>
397,LA,"West Baton Rouge, Iberville",22,<NA>
400,LA,"West Baton Rouge, Iberville",22,<NA>
482,TX,"Midland, Glasscock",48,<NA>
513,TX,TBD,48,<NA>
519,TX,<NA>,48,<NA>


### Type Validation
All the ID columns and the coordinates are CSV string arrays that need parsing and conversion to numeric.
### Cross-Field Validation - Defer
A thorough cleaning would involve geocoding the given coordinates and making sure they match the given state, county values. Also reverse geocoding the given street address and computing distance vs given coordinates. But I'll defer that until we actually do something with the lat, lon values.